In [1]:
import re
import pandas as pd
from pykospacing import Spacing

from konlpy.tag import Kkma
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
spacing = Spacing()

def preprocessing(text):                                                                # CountVectorizer는 문장 형태로 가져와야 함.
    return text

def get_stopwords():                                                                    # 불용어 리스트 가져오기
    stopwords = set()
    with open('../../../../../data/stopwords-ko.txt', 'r', encoding='utf-8') as file:
        for line in file:
            stopwords.add(line.strip())
    return stopwords

def morph_and_remove_stopwords(text, stopwords):                                        # 형태소 분석 및 불용어 제거 함수
    text = preprocessing(text)                                                          # 전처리
    kkma = Kkma()                                                                       # Kkma 형태소 분석기 초기화
    morphs = kkma.morphs(text)
    meaningful_words = [word for word in morphs if word not in stopwords]               # 불용어 제거
    return meaningful_words

In [6]:
stopwords = get_stopwords()                                                             # 불용어 리스트 가져오기
df = pd.read_excel('../../../../../data/filtered_news.xlsx')             # 분석할 파일 불러오기.
# df = df[:100]
# 데이터프레임의 'feature' 열에 대해 형태소 분석 및 불용어 제거 적용
df['processed_feature'] = df['feature'].apply(lambda x: ' '.join(morph_and_remove_stopwords(x, stopwords)))

In [7]:
vectorizer = CountVectorizer() 

X = vectorizer.fit_transform(df['processed_feature'])

bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df_bow = pd.concat([df, bow_df], axis=1)                                                # 원본 데이터프레임에 벡터화된 특성 추가